In [1]:
import trimesh
import datajoint as dj
import numpy as np
import time

In [ ]:
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

In [ ]:
key = dict(segment_id=648518346349496405,segmentation=3)

#get the full mesh
verts,faces = (pinky.ExcitatoryLeftoverMeshes & key).fetch1("vertices","triangles")

#get the labels
verts_labels,faces_labels = (pinky.LeftoverCoarseLabelFinal & key).fetch1("vertices","triangles")
verts_labels

#get the number of unique labels:
unique_labels = np.unique(verts_labels)
print(unique_labels)

#create the Trimesh of the whole mesh
total_mesh = trimesh.Trimesh(verts,faces,process=False)



In [ ]:
#what do we want in order to get original vertices and triangles:
"""
Want to know the mapping of the submeshes vertices and faces
back to the indicies of the original

For submesh:
traingles = from the initial face label indices
vertices = find from the faces

For the split method:
Use the faces from the connected components

components = connected_components(edges=adjacency,
                                  nodes=np.arange(len(mesh.faces)),
                                  min_len=min_len,
                                  engine=engine)
meshes = mesh.submesh(components,
                      only_watertight=only_watertight)
return meshes



"""

In [ ]:
celii_submesh(original_vertices,original_faces,faces_sequence)
# avoid nuking the cache on the original mesh
original_faces = mesh.faces.view(np.ndarray)
original_vertices = mesh.vertices.view(np.ndarray)

faces = []
vertices = []
normals = []
visuals = []

# for reindexing faces
mask = np.arange(len(original_vertices))

for faces_index in faces_sequence:
    # sanitize indices in case they are coming in as a set or tuple
    faces_index = np.asanyarray(faces_index, dtype=np.int64)
    if len(faces_index) == 0:
        continue
    faces_current = original_faces[faces_index]
    unique = np.unique(faces_current.reshape(-1))

    # redefine face indices from zero
    mask[unique] = np.arange(len(unique))
    normals.append(mesh.face_normals[faces_index])
    faces.append(mask[faces_current])
    vertices.append(original_vertices[unique])
    visuals.append(mesh.visual.face_subset(faces_index))

In [ ]:
def split(mesh,
          only_watertight=True,
          adjacency=None,
          engine=None):
    """
    Split a mesh into multiple meshes from face connectivity.
    If only_watertight is true, it will only return watertight meshes
    and will attempt single triangle/quad repairs.
    Parameters
    ----------
    mesh: Trimesh
    only_watertight: if True, only return watertight components
    adjacency: (n,2) list of face adjacency to override using the plain
               adjacency calculated automatically.
    engine: str, which engine to use. ('networkx', 'scipy', or 'graphtool')
    Returns
    ----------
    meshes: list of Trimesh objects
    """

    if adjacency is None:
        adjacency = mesh.face_adjacency

    # if only watertight the shortest thing we can split has 3 triangles
    if only_watertight:
        min_len = 3
    else:
        min_len = 1

    components = connected_components(edges=adjacency,
                                      nodes=np.arange(len(mesh.faces)),
                                      min_len=min_len,
                                      engine=engine)